# Common Unconstrained Optimization Methods

To start with, we will implement the functions to calculate gradient vector and hessian matrix of a given function precisely.

Let's set a math func for testing later functions first.

In [93]:
from sympy import Symbol, hessian, diff, Matrix
import numpy as np


x = [Symbol(f'x{i}') for i in range(1, 4)]
x0 = [1, 0, 1]
func = x[0]**4 + 2* x[1]**3 + 3* x[2]**2 - x[0]**2 * x[1] + 4* x[1]* x[2] - x[0]* x[2]**2 
func

x1**4 - x1**2*x2 - x1*x3**2 + 2*x2**3 + 4*x2*x3 + 3*x3**2

Here is our function to calculate the gradient vector.

In [87]:
# FIXME: here is a bug -- if mode is "symbol", there's no need to send x0 

def gdt(func, vars, x0, mode="value"):
    """Calculate gradient of func at x0
       mode: value, symbol"""
    grad = [diff(func, xi) for xi in vars]
    grad_symbol = Matrix([gd for gd in grad])
    
    n = len(vars)
    X0 = {xi: xi_ for xi, xi_ in zip(vars, x0)}
    grad_subs = [g.subs(X0) for g in grad]
    grad_value = np.array([g.evalf() for g in grad_subs], dtype=float).reshape(n, 1)

    if mode == "value":
        return grad_value
    if mode == "symbol":
        return grad_symbol
    if True:
        return "No such mode!"

In [88]:
gdt(func, x, x0, mode="value")

array([[3.],
       [3.],
       [4.]])

In [89]:
gdt(func, x, x0, mode="symbol")

Matrix([
[4*x1**3 - 2*x1*x2 - x3**2],
[  -x1**2 + 6*x2**2 + 4*x3],
[   -2*x1*x3 + 4*x2 + 6*x3]])

Here is our function to calculate the hessian matrix.

In [90]:
# FIXME: here is a bug -- if mode is "symbol", there's no need to send x0 

def hess(func, vars, x0, mode="value"):
    """Calculate hessisan matrix of func at x0
       mode: value, symbol"""
    hs = hessian(func, vars)
    hs_symbol = hs
    
    n = len(vars)
    X0 = {xi: xi_ for xi, xi_ in zip(vars, x0)}
    hs_subs = [fs.subs(X0) for fs in hs]
    hs_value = np.array([fs.evalf() for fs in hs_subs], dtype=float).reshape(n, n)

    if mode == "value":
        return hs_value
    if mode == "symbol":
        return hs_symbol
    if True:
        return "No such mode!"

In [91]:
hess(func, x, x0, mode="value")

array([[12., -2., -2.],
       [-2.,  0.,  4.],
       [-2.,  4.,  4.]])

In [92]:
hess(func, x, x0, mode="symbol")

Matrix([
[12*x1**2 - 2*x2, -2*x1,    -2*x3],
[          -2*x1, 12*x2,        4],
[          -2*x3,     4, 6 - 2*x1]])